In [1]:
%load_ext line_profiler

from pathlib import Path
import sys

import numpy as np

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])
print(parent_dir)
# Add to sys.path

path_set = set(sys.path)
if parent_dir not in path_set:
    sys.path.insert(0, parent_dir)

print(sys.path)

/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline
['/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/colinbull/Library/Caches/pypoetry/virtualenvs/fbit-data-pipeline-aJYNke-B-py3.12/lib/python3.12/site-packages']


# Metric RAG

* median, decile, status (Red / Amber / Green), percentage
* All cost catgeories - cost per pupil
* Utilities and Premises and staff - per m^2

In [2]:
import pandas as pd
import src.pipeline.rag as rag
import glob 
import os
import logging
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

# Create and clean directory
from pathlib import Path
Path("output/metric-rag").mkdir(parents=True, exist_ok=True)

files = glob.glob("output/metric-rag/*")
for f in files:
    os.remove(f)

## Loading saved comparator sets

In [3]:
raw_data = pd.read_parquet('output/comparator-sets/academies.parquet')
raw_comparators = pd.read_parquet('output/comparator-sets/academy_comparators.parquet')

In [4]:
cols = raw_data.columns.isin(rag.base_cols) | raw_data.columns.str.endswith("_Per Unit")
raw_data[raw_data.columns[cols]]

,Number of pupils,OfstedRating (name),Percentage Free school meals,Percentage SEN,Total Internal Floor Area,Age Average Score,Teaching and Teaching support staff_Teaching staff_Per Unit,Teaching and Teaching support staff_Supply teaching staff_Per Unit,Teaching and Teaching support staff_Education support staff_Per Unit,Teaching and Teaching support staff_Educational consultancy_Per Unit,...,Other costs_Staff development and training_Per Unit,Other costs_Staff-related insurance_Per Unit,Other costs_Supply teacher insurance_Per Unit,Other costs_Rent and rates_Per Unit,Other costs_Special facilities_Per Unit,Other costs_Other insurance premiums_Per Unit,Other costs_Interest charges for loan and bank_Per Unit,Other costs_Direct revenue financing (revenue contributions to capital)_Per Unit,Other costs_PFI charges_Per Unit,Other costs_Total_Per Unit
URN,,,,,,,,,,,,,,,,,,,,,
101849,1419,Good,11.3,3.435934,27058.0,43.834208,3977.448908,0.704722,916.138125,23.960536,...,52.854123,0.0,0.0,55.673009,0.0,31.007752,195.912615,250.17618,0.0,587.737844
105135,1160,Requires improvement,29.4,2.500000,11352.0,16.7463,5134.482759,0.0,865.517241,156.034483,...,100.0,0.0,0.0,70.689655,0.0,23.275862,0.0,65.517241,0.0,259.482759
108420,1449,Outstanding,15.3,0.692521,15903.0,37.0,4222.222222,0.0,466.52864,0.690131,...,21.394065,0.0,0.0,34.506556,0.0,24.84472,0.0,91.097308,0.0,206.349206
123627,1567,Good,23.6,0.129450,28298.0,23.204891,4039.56605,0.0,672.622846,24.25016,...,41.480536,0.0,0.0,37.013401,0.0,31.269943,0.0,712.188896,0.0,841.097639
129342,904,Good,47.0,2.447164,17294.0,16.930612,3628.318584,0.0,808.628319,86.283186,...,7.743363,0.0,0.0,51.99115,3.318584,88.495575,0.0,0.0,0.0,254.424779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149222,0,,39.8,3.455285,2850.0,50.042394,inf,NaN,inf,inf,...,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inf
149222,0,,39.8,3.455285,2850.0,50.042394,inf,NaN,inf,inf,...,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inf
149299,0,,31.4,3.944316,9429.0,41.71524,inf,NaN,inf,inf,...,NaN,NaN,NaN,inf,NaN,NaN,NaN,NaN,NaN,inf


In [5]:
comps = raw_comparators.loc[135665]["Pupil"]
series = raw_data[raw_data.index.isin(comps)]

(df, categories) = rag.get_category_cols_predicates("Teaching and Teaching support staff", series)
categories

array(['Teaching and Teaching support staff_Teaching staff_Per Unit',
       'Teaching and Teaching support staff_Supply teaching staff_Per Unit',
       'Teaching and Teaching support staff_Education support staff_Per Unit',
       'Teaching and Teaching support staff_Educational consultancy_Per Unit',
       'Teaching and Teaching support staff_Agency supply teching staff_Per Unit',
       'Teaching and Teaching support staff_Total_Per Unit'], dtype=object)

Compute the rag calculation for the cost categories given in the [Benchmarking cost categories doc](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/DfEFinancialBenchmarking/_layouts/15/Doc.aspx?sourcedoc=%7B622FB0F9-7CB1-445A-8FFA-664F8857F036%7D&file=Benchmarking%20cost%20categories%20and%20sub-categories.docx&action=default&mobileredirect=true)

In [6]:
rs = rag.compute_rag(raw_data, raw_comparators)
pd.DataFrame(rs)

2024-05-18 16:46:03,275 | INFO : Completed 100 RAGs in 1.57 secs
2024-05-18 16:46:04,662 | INFO : Completed 200 RAGs in 1.39 secs
2024-05-18 16:46:06,027 | INFO : Completed 300 RAGs in 1.36 secs
2024-05-18 16:46:07,475 | INFO : Completed 400 RAGs in 1.45 secs
2024-05-18 16:46:08,845 | INFO : Completed 500 RAGs in 1.37 secs
2024-05-18 16:46:10,206 | INFO : Completed 600 RAGs in 1.36 secs
2024-05-18 16:46:11,570 | INFO : Completed 700 RAGs in 1.36 secs
2024-05-18 16:46:13,123 | INFO : Completed 800 RAGs in 1.55 secs
2024-05-18 16:46:14,488 | INFO : Completed 900 RAGs in 1.36 secs
2024-05-18 16:46:15,864 | INFO : Completed 1000 RAGs in 1.38 secs
2024-05-18 16:46:17,216 | INFO : Completed 1100 RAGs in 1.35 secs
2024-05-18 16:46:18,583 | INFO : Completed 1200 RAGs in 1.37 secs
2024-05-18 16:46:19,940 | INFO : Completed 1300 RAGs in 1.36 secs
2024-05-18 16:46:21,297 | INFO : Completed 1400 RAGs in 1.36 secs
2024-05-18 16:46:22,664 | INFO : Completed 1500 RAGs in 1.37 secs
2024-05-18 16:46:24

--- Logging error ---
Traceback (most recent call last):
  File "/Users/colinbull/Library/Caches/pypoetry/virtualenvs/fbit-data-pipeline-aJYNke-B-py3.12/lib/python3.12/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 2606, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 2630, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 147756

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline/src/pipeline/rag.py", line 192, in compute_rag
    pupil_urns = comparators["Pupil"][urn]
         

,Urn,Category,Sub category,Value,Mean,Diff Mean,Key,Percentage Diff,Percentile,Decile,Rag
0,101849,Teaching and Teaching support staff,Teaching staff,3977.448908,3738.334089,239.114819,other,6.011763,89.000000,8.900000,amber
1,101849,Teaching and Teaching support staff,Supply teaching staff,0.704722,0.352361,0.352361,other,50.000000,52.333333,5.233333,green
2,101849,Teaching and Teaching support staff,Education support staff,916.138125,628.582340,287.555786,other,31.387820,82.333333,8.233333,amber
3,101849,Teaching and Teaching support staff,Educational consultancy,23.960536,13.599288,10.361248,other,43.242972,62.333333,6.233333,amber
4,101849,Teaching and Teaching support staff,Agency supply teching staff,377.026075,71.548317,305.477758,other,81.022979,99.000000,9.900000,red
...,...,...,...,...,...,...,...,...,...,...,...
369795,147755,Other costs,Other insurance premiums,0.000000,21.686747,-21.686747,other,0.000000,22.333333,2.233333,green
369796,147755,Other costs,Interest charges for loan and bank,0.000000,0.000000,0.000000,other,0.000000,99.000000,9.900000,red
369797,147755,Other costs,Direct revenue financing (revenue contribution...,2.500000,0.000000,2.500000,other,100.000000,59.000000,5.900000,amber
369798,147755,Other costs,PFI charges,0.000000,0.000000,0.000000,other,0.000000,99.000000,9.900000,red


In [ ]:
%lprun -f rag.find_percentile list(rag.compute_rag(raw_data.head(10), raw_comparators))